In [18]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

CSV_PATH = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/data/stitched_detail_data/RD_LFP_HPPC_REPT_Group0_150_0001_10_100.csv")
VALID_STEPS = {"CC_Chg", "CC_DChg"}  # treat only the short CC pulses as valid

df = pd.read_csv(CSV_PATH)
df["Absolute time"] = pd.to_datetime(df["Absolute time"])
df["time_s"] = (df["Absolute time"] - df["Absolute time"].min()).dt.total_seconds()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time_s"], y=df["volt(V)"], mode="lines", name="Voltage", line=dict(color="#4cc3ff")))

band_colors = {"CC_Chg": "rgba(255, 215, 0, 0.25)", "CC_DChg": "rgba(0, 191, 255, 0.25)"}

skip_first_cccv = False
for (cycle, step), grp in df.groupby(["Cycle No", "Step No"], sort=False):
    step_name = str(grp["Step name"].iloc[0]).strip()

    if step_name == "CCCV_Chg" and not skip_first_cccv:
        skip_first_cccv = True
        continue
    if step_name == "CC_DChg" and step == 47:   # skip long tail
        continue
    if step_name not in VALID_STEPS:
        continue

    start_t = grp["time_s"].iloc[0]
    end_t = grp["time_s"].iloc[-1]
    local_v = grp["volt(V)"]

    fig.add_shape(
        type="rect",
        xref="x", yref="y",
        x0=start_t, x1=end_t,
        y0=local_v.min(), y1=local_v.max(),
        fillcolor=band_colors.get(step_name, "rgba(200,200,200,0.2)"),
        line=dict(width=0),
        layer="below",
    )
    fig.add_annotation(
        x=(start_t + end_t) / 2,
        y=local_v.max(),
        text=f"Step {step} ({step_name})",
        showarrow=False,
        font=dict(size=10, color="white"),
        yshift=12,
    )

fig.update_layout(
    title=CSV_PATH.stem,
    xaxis_title="Time (s)",
    yaxis_title="Voltage (V)",
    template="plotly_dark",
    height=500,
)

fig.show()


In [1]:
from utils.flat_window_optimizer import optimize_flat_parameters


In [2]:
optimize_flat_parameters()

Best flat_window=3, flat_threshold=0.005, RMSE=0.01041259512258746


In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

CELL = "RD_LFP_HPPC_REPT_Group0_150_0001_10_100"

base_dir = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/data/optimiser_results")
summary_path = base_dir / "resistances" / f"{CELL}_resistance_summary.csv"
deriv_dir = base_dir / "voltage_derivatives"
raw_path = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/data/stitched_detail_data") / f"{CELL}.csv"

summary = pd.read_csv(summary_path)
summary["pulse_no"] = range(1, len(summary) + 1)

raw = pd.read_csv(raw_path)
raw["Absolute time"] = pd.to_datetime(raw["Absolute time"])
raw["time_s"] = (raw["Absolute time"] - raw["Absolute time"].min()).dt.total_seconds()

for _, row in summary.iterrows():
    pulse_id = f"cycle{int(row['cycle'])}_step{int(row['step'])}"
    deriv_df = pd.read_csv(deriv_dir / f"{CELL}_{pulse_id}_derivatives.csv")

    mask = (raw["time_s"] >= row["t_pre"] - 1) & (raw["time_s"] <= row["t_end"] + 1)
    seg = raw.loc[mask].sort_values("time_s")

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=seg["time_s"], y=seg["volt(V)"], mode="markers+lines", name="Voltage"),
                  secondary_y=False)
    fig.add_trace(go.Scatter(x=deriv_df["time_s"], y=deriv_df["dv_dt"],
                             mode="lines", name="dV/dt", line=dict(color="orange")),
                  secondary_y=True)
    fig.add_trace(go.Scatter(x=deriv_df["time_s"], y=deriv_df["d2v_dt2"],
                             mode="lines", name="d²V/dt²", line=dict(color="magenta")),
                  secondary_y=True)

    markers = [
        ("V_pre", row["t_pre"], row["V_pre"], "green"),
        ("V_post", row["t_post"], row["V_post"], "cyan"),
        ("V_fast", row["t_fast"], row["V_fast"], "yellow"),
        ("V_end", row["t_end"], row["V_end"], "lime"),
    ]
    for label, tx, vy, color in markers:
        fig.add_trace(go.Scatter(x=[tx], y=[vy], mode="markers+text",
                                 text=[label], textposition="bottom center",
                                 marker=dict(color=color, size=8), name=label),
                      secondary_y=False)

    fig.update_layout(
        title=f"Pulse {int(row['pulse_no'])} (cycle {int(row['cycle'])}, step {int(row['step'])})",
        xaxis_title="Time (s)",
        template="plotly_dark",
        height=500,
    )
    fig.update_yaxes(title_text="Voltage (V)", secondary_y=False)
    fig.update_yaxes(title_text="dV/dt, d²V/dt²", secondary_y=True)
    fig.show()


In [4]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

CELL = "RD_LFP_HPPC_REPT_Group0_150_0091_10_100"
summary_path = Path(f"/home/kcv/Desktop/HPPC_post_processing/post_processing/data/optimiser_results/resistances/{CELL}_resistance_summary.csv")
summary = pd.read_csv(summary_path)
summary["pulse_no"] = range(1, len(summary) + 1)

fig = go.Figure()
for col, color in zip(["R0", "R1", "R2"], ["#1f77b4", "#ff7f0e", "#2ca02c"]):
    fig.add_trace(
        go.Scatter(
            x=summary["pulse_no"],
            y=summary[col].abs(),
            mode="lines+markers",
            name=col,
            line=dict(color=color),
        )
    )

fig.update_layout(
    title=f"Resistance vs Pulse Number — {summary['file'].iloc[0]}",
    xaxis_title="Pulse number",
    yaxis_title="Resistance (Ω)",
    template="plotly_dark",
)
fig.show()


In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

CELL = "RD_LFP_HPPC_REPT_Group0_150_0091_10_100"
summary_path = Path(f"/home/kcv/Desktop/HPPC_post_processing/post_processing/data/optimiser_results/resistances/{CELL}_resistance_summary.csv")
raw_path = Path(f"/home/kcv/Desktop/HPPC_post_processing/post_processing/data/stitched_detail_data/{CELL}.csv")

summary = pd.read_csv(summary_path)
summary["pulse_no"] = range(1, len(summary) + 1)

raw = pd.read_csv(raw_path)
raw["Absolute time"] = pd.to_datetime(raw["Absolute time"])
raw["time_s"] = (raw["Absolute time"] - raw["Absolute time"].min()).dt.total_seconds()

for _, row in summary.iterrows():
    mask = (raw["time_s"] >= row["t_post"]) & (raw["time_s"] <= row["t_end"])
    seg = raw.loc[mask]
    if seg.empty or row["duration_post_fast"] <= 0 or row["duration_fast_end"] <= 0:
        continue

    t_rel = seg["time_s"].to_numpy() - row["t_post"]
    delta_measured = row["V_pre"] - seg["volt(V)"].to_numpy()
    delta_model = row["I_step"] * (
        row["R0"]
        + row["R1"] * (1 - np.exp(-t_rel / row["duration_post_fast"]))
        + row["R2"] * (1 - np.exp(-t_rel / row["duration_fast_end"]))
    )
    error = delta_measured - delta_model
    rmse = np.sqrt(np.mean(error**2))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t_rel, y=delta_measured, mode="lines", name="ΔV measured"))
    fig.add_trace(go.Scatter(x=t_rel, y=delta_model, mode="lines", name="ΔV model"))
    fig.add_trace(go.Scatter(x=t_rel, y=error, mode="lines", name="Error", line=dict(dash="dot")))
    fig.update_layout(
        title=f"Pulse {int(row['pulse_no'])} ΔV(t) — RMSE = {rmse*1e3:.2f} mV",
        xaxis_title="Time since pulse start (s)",
        yaxis_title="ΔV (V)",
        template="plotly_dark",
    )
    fig.show()


# Plotting resistances and capacitances for all cells

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path

resist_dir = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/data/optimiser_results/resistances")
plots_dir = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/plots")
plots_dir.mkdir(parents=True, exist_ok=True)
pdf_path = plots_dir / "resistance_vs_pulse.pdf"

colors = {"R0": "#1f77b4", "R1": "#ff7f0e", "R2": "#2ca02c"}

with PdfPages(pdf_path) as pdf:
    for csv_path in sorted(resist_dir.glob("*_resistance_summary.csv")):
        summary = pd.read_csv(csv_path)
        if summary.empty:
            continue

        summary["pulse_no"] = range(1, len(summary) + 1)
        summary["soc_pct"] = (summary["pulse_no"] * 10).clip(upper=100)

        fig, ax = plt.subplots(figsize=(10, 5))
        for col in ["R0", "R1", "R2"]:
            ax.plot(
                summary["pulse_no"],
                summary[col].abs(),
                marker="o",
                linewidth=2,
                label=f"{col} (|Ω|)",
                color=colors[col],
            )

        ax.set_title(f"Resistance vs Pulse Number — {summary['file'].iloc[0]}", fontsize=13)
        ax.set_xlabel("Pulse number")
        ax.set_ylabel("Resistance (Ω)")
        ax.grid(True, alpha=0.35)
        ax.legend(loc="upper right")

        ax_soc = ax.twiny()
        ax_soc.set_xlim(ax.get_xlim())
        ax_soc.set_xticks(summary["pulse_no"])
        ax_soc.set_xticklabels(summary["soc_pct"].astype(int))
        ax_soc.set_xlabel("State of Charge (%) — pulse 1 = 10% SOC")

        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)

print(f"Saved combined PDF to {pdf_path}")


Saved combined PDF to /home/kcv/Desktop/HPPC_post_processing/post_processing/plots/resistance_vs_pulse.pdf


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path

resist_dir = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/data/optimiser_results/resistances")
plots_dir = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/plots")
plots_dir.mkdir(parents=True, exist_ok=True)
pdf_path = plots_dir / "capacitance_vs_pulse.pdf"

colors = {"C1": "#ff7f0e", "C2": "#2ca02c"}

with PdfPages(pdf_path) as pdf:
    for csv_path in sorted(resist_dir.glob("*_resistance_summary.csv")):
        summary = pd.read_csv(csv_path)
        if summary.empty or not {"C1", "C2"}.issubset(summary.columns):
            continue

        summary["pulse_no"] = range(1, len(summary) + 1)
        summary["soc_pct"] = (summary["pulse_no"] * 10).clip(upper=100)

        fig, ax = plt.subplots(figsize=(10, 5))
        for col in ["C1", "C2"]:
            ax.plot(
                summary["pulse_no"],
                summary[col],
                marker="o",
                linewidth=2,
                label=f"{col} (F)",
                color=colors[col],
            )

        ax.set_title(f"Capacitance vs Pulse Number — {summary['file'].iloc[0]}", fontsize=13)
        ax.set_xlabel("Pulse number")
        ax.set_ylabel("Capacitance (F)")
        ax.grid(True, alpha=0.35)
        ax.legend(loc="upper right")

        ax_soc = ax.twiny()
        ax_soc.set_xlim(ax.get_xlim())
        ax_soc.set_xticks(summary["pulse_no"])
        ax_soc.set_xticklabels(summary["soc_pct"].astype(int))
        ax_soc.set_xlabel("State of Charge (%) — pulse 1 = 10% SOC")

        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)

print(f"Saved combined PDF to {pdf_path}")


Saved combined PDF to /home/kcv/Desktop/HPPC_post_processing/post_processing/plots/capacitance_vs_pulse.pdf


### Voltage end points identification for 2RC and error in delta V

In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from io import BytesIO
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from PIL import Image

optim_dir = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/data/optimiser_results")
resist_dir = optim_dir / "resistances"
deriv_dir = optim_dir / "voltage_derivatives"
raw_dir = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/data/stitched_detail_data")
out_pdf = Path("/home/kcv/Desktop/HPPC_post_processing/post_processing/plots/pulse_diagnostics.pdf")
out_pdf.parent.mkdir(parents=True, exist_ok=True)

with PdfPages(out_pdf) as pdf:
    for summary_path in sorted(resist_dir.glob("*_resistance_summary.csv")):
        summary = pd.read_csv(summary_path)
        if summary.empty:
            continue

        cell = summary_path.stem.replace("_resistance_summary", "")
        raw = pd.read_csv(raw_dir / f"{cell}.csv")
        raw["Absolute time"] = pd.to_datetime(raw["Absolute time"])
        raw["time_s"] = (raw["Absolute time"] - raw["Absolute time"].min()).dt.total_seconds()
        summary["pulse_no"] = range(1, len(summary) + 1)

        for _, row in summary.iterrows():
            pulse_id = f"cycle{int(row['cycle'])}_step{int(row['step'])}"
            deriv_path = deriv_dir / f"{cell}_{pulse_id}_derivatives.csv"
            if not deriv_path.exists():
                continue
            deriv_df = pd.read_csv(deriv_path)

            interval_mask = (raw["time_s"] >= row["t_pre"]) & (raw["time_s"] <= row["t_end"])
            seg = raw.loc[interval_mask].sort_values("time_s")
            if seg.empty or row["duration_post_fast"] <= 0 or row["duration_fast_end"] <= 0:
                continue

            deriv_mask = (deriv_df["time_s"] >= row["t_pre"]) & (deriv_df["time_s"] <= row["t_end"])
            deriv_sub = deriv_df.loc[deriv_mask].copy()
            if deriv_sub.empty:
                continue

            # relative times (seconds from V_pre)
            seg = seg.assign(t_rel=seg["time_s"] - row["t_pre"])
            deriv_sub = deriv_sub.assign(t_rel=deriv_sub["time_s"] - row["t_pre"])
            t_end_rel = row["t_end"] - row["t_pre"]
            t_post_rel = row["t_post"] - row["t_pre"]
            t_fast_rel = row["t_fast"] - row["t_pre"]

            seg_full = seg[seg["time_s"] >= row["t_post"]]
            t_rel_pulse = seg_full["time_s"].to_numpy() - row["t_post"]
            delta_measured = row["V_pre"] - seg_full["volt(V)"].to_numpy()
            delta_model = row["I_step"] * (
                row["R0"]
                + row["R1"] * (1 - np.exp(-t_rel_pulse / row["duration_post_fast"]))
                + row["R2"] * (1 - np.exp(-t_rel_pulse / row["duration_fast_end"]))
            )
            error = delta_measured - delta_model
            rmse = np.sqrt(np.mean(error**2))

            fig = make_subplots(
                rows=2,
                cols=1,
                specs=[[{"secondary_y": True}], [{"secondary_y": False}]],
                shared_xaxes=False,
                vertical_spacing=0.08,
            )

            fig.add_trace(
                go.Scatter(x=seg["t_rel"], y=seg["volt(V)"], mode="lines+markers", name="Voltage"),
                row=1, col=1, secondary_y=False,
            )
            fig.add_trace(
                go.Scatter(x=deriv_sub["t_rel"], y=deriv_sub["dv_dt"],
                           mode="lines", name="dV/dt", line=dict(color="orange")),
                row=1, col=1, secondary_y=True,
            )
            fig.add_trace(
                go.Scatter(x=deriv_sub["t_rel"], y=deriv_sub["d2v_dt2"],
                           mode="lines", name="d²V/dt²", line=dict(color="magenta")),
                row=1, col=1, secondary_y=True,
            )

            markers = [
                ("V_pre", 0.0, row["V_pre"], "green", "Δt = 0 s"),
                ("V_post", t_post_rel, row["V_post"], "cyan", f"Δt = {row['duration_pre_post']:.3f} s"),
                ("V_fast", t_fast_rel, row["V_fast"], "yellow", f"Δt = {row['duration_post_fast']:.3f} s"),
                ("V_end", t_end_rel, row["V_end"], "lime", f"Δt = {row['duration_fast_end']:.3f} s"),
            ]
            for label, tx, vy, color, text in markers:
                fig.add_trace(
                    go.Scatter(
                        x=[tx], y=[vy], mode="markers+text",
                        text=[f"{label} ({text})"], textposition="bottom center",
                        marker=dict(color=color, size=8), name=f"{label} ({text})",
                    ),
                    row=1, col=1, secondary_y=False,
                )

            fig.add_trace(go.Scatter(x=t_rel_pulse, y=delta_measured, mode="lines", name="ΔV measured"), row=2, col=1)
            fig.add_trace(go.Scatter(x=t_rel_pulse, y=delta_model, mode="lines", name="ΔV model"), row=2, col=1)
            fig.add_trace(go.Scatter(x=t_rel_pulse, y=error, mode="lines", name="Error", line=dict(dash="dot")), row=2, col=1)

            fig.update_xaxes(title_text="Time since V_pre (s)", range=[0, max(t_end_rel, 1e-3)], row=1, col=1)
            fig.update_xaxes(title_text="Time since pulse start (s)", row=2, col=1)
            fig.update_yaxes(title_text="Voltage (V)", row=1, col=1, secondary_y=False)
            fig.update_yaxes(title_text="dV/dt, d²V/dt²", row=1, col=1, secondary_y=True)
            fig.update_yaxes(title_text="ΔV (V)", row=2, col=1)
            fig.update_layout(
                title=f"{cell} — Pulse {int(row['pulse_no'])} (cycle {int(row['cycle'])}, step {int(row['step'])}) | RMSE={rmse*1e3:.2f} mV",
                template="plotly_dark",
                height=720,
            )

            png = fig.to_image(format="png", width=1100, height=780)
            img = Image.open(BytesIO(png))
            mpl_fig = plt.figure(figsize=(img.width / 100, img.height / 100))
            plt.axis("off")
            plt.imshow(img)
            pdf.savefig(mpl_fig, bbox_inches="tight")
            plt.close(mpl_fig)

print(f"Saved diagnostic PDF to {out_pdf}")


Saved diagnostic PDF to /home/kcv/Desktop/HPPC_post_processing/post_processing/plots/pulse_diagnostics.pdf
